In [1]:
import os

In [2]:
%pwd

'd:\\Deep-Learning-Project\\Deepl-Learning-CNN-Project\\research'

In [3]:
os.chdir('../')


In [4]:
%pwd

'd:\\Deep-Learning-Project\\Deepl-Learning-CNN-Project'

In [13]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

#### update components



In [14]:
from src.CNN_Classification.constants import *
from src.CNN_Classification.utils.common import read_yaml, create_directories
import tensorflow as tf

In [15]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "MRI DATASET") 
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [16]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [ ]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,  # Normalize pixel values
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],  #(224,224)
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
            class_mode = 'sparse'   # Since we're using SparseCategoricalCrossentropy
        )

        # Validation and test generators (NO augmentation)
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs)
        test_datagenerator =  tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs)

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.validation_data,  # Points to validation folder
            shuffle=False,
            **dataflow_kwargs
        )

        self.test_generator = test_datagenerator.flow_from_directory(
            directory=self.config.test_data,  #  Points to test folder
            shuffle=False,
            **dataflow_kwargs
        )
         # Train generator (WITH augmentation if enabled)

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.train_data,
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

##### update pipeline

In [22]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2025-03-16 11:28:04,085: INFO: common: yaml file:config\config.yaml loaded successfully]
[2025-03-16 11:28:04,089: INFO: common: yaml file:params.yaml loaded successfully]
[2025-03-16 11:28:04,090: INFO: common: created directory at: artifacts]
[2025-03-16 11:28:04,092: INFO: common: created directory at: artifacts\training]


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'artifacts\prepare_base_model\base_model_updated.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)